In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

%matplotlib inline

In [5]:
data = pd.read_excel('tim_data/tim_all_cofactors.xlsx')
data = data.assign(Done=[False for i in range(len(data))])
data.head()

,name,element_id,ecocyc_id,plot_id,count,parent,done,Done
0,FE,FE,NaN,10001.0,483709.0,-1,0.0,False
1,Electron Transfer Chains,FE,Electron-Transfer,10104.0,116919.0,10001,0.0,False
2,Amino Acid Biosynthesis,FE,Amino-Acid-Biosynthesis,10109.0,38957.0,10001,0.0,False
3,TCA Cycle,FE,TCA-VARIANTS,10103.0,35260.0,10001,0.0,False
4,Regulation of Transcription,FE,Regulation of transcription,10113.0,33935.0,10001,0.0,False


In [14]:
def sunburst_remaining(nodes, total=np.pi * 2, offset=0, level=0, ax=None, defined_color=None):
    ax = ax or plt.subplot(111, projection='polar')

    if level == 0 and len(nodes) == 1:
        label, value, subnodes = nodes[0]
        ax.bar([0], [0.5], [np.pi * 2])
        ax.text(0, 0, label, ha='center', va='center')
        sunburst_remaining(subnodes, total=value, level=level + 1, ax=ax)
    elif nodes:
        d = np.pi * 2 / total
        labels, widths, colors = [], [], []
        local_offset = offset
        color_index = 0
        remaining_value = 0
        non_negligible_nodes_length = 0
        for label, value, subnodes in nodes:
            if value * d >= 0.1: 
                non_negligible_nodes_length += 1
                labels.append(label)
                widths.append(value * d)
                if label == '':
                    colors.append('black')
                elif level == 1:
                    colors.append(set2[color_index])
                    sunburst_remaining(subnodes, total=total, offset=local_offset, level=level + 1, ax=ax, defined_color=color_index)
                else:
                    if color_index % 2 == 0:
                        colors.append(pastel_set2[defined_color])
                    else:
                        colors.append(set2[defined_color])
                    sunburst_remaining(subnodes, total=total, offset=local_offset, level=level + 1, ax=ax, defined_color=defined_color)
                color_index += 1
                local_offset += value
            else:
                remaining_value += value
        
        if remaining_value > 0:
            labels.append('Remaining')
            widths.append(remaining_value * d)
            
            if level == 1:
                colors.append(set2[color_index])
                sunburst_remaining(subnodes, total=total, offset=local_offset, level=level + 1, ax=ax, defined_color=color_index)
            else:
                if color_index % 2 == 0:
                    colors.append(pastel_set2[defined_color])
                else:
                    colors.append(set2[defined_color])
                sunburst_remaining(subnodes, total=total, offset=local_offset, level=level + 1, ax=ax, defined_color=defined_color)
            color_index += 1
            local_offset += remaining_value
        
        values = np.cumsum([offset * d] + widths[:-1])
        heights = [1] * (non_negligible_nodes_length + 1)
        bottoms = np.zeros(non_negligible_nodes_length + 1) + level - 0.5
        rects = ax.bar(values, heights, widths, bottoms, linewidth=0.2,
                       edgecolor='white', align='edge', color=colors)
        for rect, label in zip(rects, labels):
            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_y() + rect.get_height() / 2
            rotation = (90 + (360 - np.degrees(x) % 180)) % 360
            ax.text(x, y, label, rotation=rotation, ha='center', va='center') 

    if level == 0:
        ax.set_theta_direction(-1)
        ax.set_theta_zero_location('N')
        ax.set_axis_off()
        
        plt.savefig('tim_data/all_sunburst.png', bbox_inches='tight')
        
def sunburst(nodes, total=np.pi * 2, offset=0, level=0, ax=None, defined_color=None):
    ax = ax or plt.subplot(111, projection='polar')

    if level == 0 and len(nodes) == 1:
        label, value, subnodes = nodes[0]
        ax.bar([0], [0.5], [np.pi * 2])
        ax.text(0, 0, label, ha='center', va='center')
        sunburst(subnodes, total=value, level=level + 1, ax=ax)
    elif nodes:
        d = np.pi * 2 / total
        labels, widths, colors = [], [], []
        local_offset = offset
        color_index = 0
        for label, value, subnodes in nodes:
            if value * d >= 0.1: 
                labels.append(label)
            widths.append(value * d)
            if label == '':
                colors.append('black')
            elif level == 1:
                colors.append(set2[color_index])
                sunburst(subnodes, total=total, offset=local_offset, level=level + 1, ax=ax, defined_color=color_index)
            else:
                if color_index % 2 == 0:
                    colors.append(pastel_set2[defined_color])
                else:
                    colors.append(set2[defined_color])
                sunburst(subnodes, total=total, offset=local_offset, level=level + 1, ax=ax, defined_color=defined_color)
            color_index += 1
            local_offset += value
        
        values = np.cumsum([offset * d] + widths[:-1])
        heights = [1] * len(nodes)
        bottoms = np.zeros(len(nodes)) + level - 0.5
        rects = ax.bar(values, heights, widths, bottoms, linewidth=1,
                       edgecolor='white', align='edge', color=colors)
        for rect, label in zip(rects, labels):
            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_y() + rect.get_height() / 2
            rotation = (90 + (360 - np.degrees(x) % 180)) % 360
            ax.text(x, y, label, rotation=rotation, ha='center', va='center') 

    if level == 0:
        ax.set_theta_direction(-1)
        ax.set_theta_zero_location('N')
        ax.set_axis_off()
        
        plt.savefig('tim_data/all_sunburst.png', bbox_inches='tight')

In [7]:
parent_layer_sums = {}
parent_list = data['parent'].unique().tolist()
# layer_list = data['Layer'].unique().tolist()
# max_layer = max(layer_list)

for parent in parent_list:
#    for layer in layer_list:
    cur_sum = 0
    for index, row in data.iterrows():
        if row['parent'] == parent: #and index != len(data) - 1:
            cur_sum += row['count']
    parent_layer_sums[parent] = cur_sum
        
print(parent_layer_sums)

{-1: 960883.0, 10001: 367831.0, 10104: 116919.0, 10109: 38957.0, 10103: 35260.0, 10113: 33935.0, 10106: 79336.0, 10102: 9919.0, 10116: 19160.0, 10108: 11929.0, 10114: 2348.0, 10107: 7921.0, 10105: 2399.0, 10115: 2383.0, 10117: 3466.0, 10112: 1934.0, 10110: 975.0, 10111: 990.0, 20001: 321084.0, 20125: 75809.0, 20103: 28023.0, 20106: 19777.0, 20124: 23753.0, 20107: 23385.0, 20113: 12363.0, 20123: 20544.0, 20116: 15298.0, 20104: 21137.0, 20110: 11982.0, 20111: 23473.0, 20105: 4804.0, 20117: 3064.0, 20118: 3064.0, 20108: 5932.0, 20126: 3502.0, 20121: 4289.0, 20129: 3060.0, 20128: 9312.0, 20112: 1087.0, 20115: 1071.0, 20114: 1027.0, 20127: 1295.0, 20109: 1644.0, 20102: 615.0, 20119: 1099.0, 20120: 415.0, 20122: 260.0, 30001: 81990.0, 30119: 47516.0, 30104: 8985.0, 30107: 6221.0, 30112: 5006.0, 30122: 7534.0, 30120: 3212.0, 30114: 2114.0, 30106: 471.0, 30117: 366.0, 30108: 113.0, 30109: 83.0, 30115: 79.0, 30123: 67.0, 30110: 60.0, 30113: 146.0, 30116: 17.0, 30103: 0.0, 30105: 0.0, 30111: 0.0

### Version that uses percentages (deprecated)

In [8]:
def transform_data_percent(df, data_tuple, parent_name, parent_value, cur_layer):
    df['Done'] = df['Done'].astype(bool)
    children = df[(df['Layer'] == cur_layer) & (df['Parent'] == parent_name) & (df['Done'] == False)]
    print(children)
    for _, child in children.iterrows():
        child_value = 0
        if parent_layer_sums[(parent_name, cur_layer)] != 0:
            child_value = child['Count'] / parent_layer_sums[(parent_name, cur_layer)] * parent_value
        child_tuple = (child['Name'], child_value, [])
        data_tuple[2].append(child_tuple)
        df.at[child.name, 'Done'] = True  # Mark as done
        transform_data_percent(df, child_tuple, child['Name'], child_value, cur_layer + 1)

## Version that uses actual count values


In [9]:
# def layer_fill(df, data_tuple, cur_layer, parent_value):
#     child_tuple = ('', parent_value, [])
#     data_tuple[2].append(child_tuple)  
#     if cur_layer != max_layer:
#         layer_fill(df, child_tuple, cur_layer + 1, parent_value)

def transform_data_count(df, data_tuple, parent_id, parent_value):
    print(data_tuple)
    df['done'] = df['done'].astype(bool)
    children = df[(df['parent'] == parent_id) & (df['done'] == False)]
    layer_sum = 0
    for _, child in children.iterrows():
        child_value = child['count']
        layer_sum += child_value
        child_tuple = (child['name'], child_value, [])
        data_tuple[2].append(child_tuple)
        df.at[child.name, 'Done'] = True  # Mark as done
        transform_data_count(df, child_tuple, child['plot_id'], child_value)

    if layer_sum != parent_value and len(data_tuple[2]) != 0:
        child_tuple = ('', parent_value - layer_sum, [])
        data_tuple[2].append(child_tuple)
        
        # if cur_layer != max_layer:
            # layer_fill(df, child_tuple, cur_layer + 1, parent_value - layer_sum)

root_value = parent_layer_sums[-1]
data_transformed = [('Root', root_value, [])]
transform_data_count(data, data_transformed[0], -1, root_value)


('Root', 960883.0, [])
('FE', 483709.0, [])
('Electron Transfer Chains', 116919.0, [])
('NADH:quinone oxidoreductase I', 40191.0, [])
('NADH:quinone oxidoreductase I,\nperipheral arm', 20031.0, [])
('succinate:quinone\noxidoreductase, iron-sulfur\ncluster binding protein', 10937.0, [])
('cytochrome\n<i>bo<sub>3</sub></i>\nubiquinol:oxygen\noxidoreductase', 11802.0, [])
('succinate:quinone\noxidoreductase', 6071.0, [])
('NADH:quinone oxidoreductase\nsubunit I', 6015.0, [])
('NADH:quinone oxidoreductase\nsubunit B', 5635.0, [])
('NADH:quinone oxidoreductase\nsubunit E', 3727.0, [])
('succinate:quinone\noxidoreductase, membrane\nprotein SdhC', 2197.0, [])
('anaerobic glycerol-3-phosphate\ndehydrogenase', 2239.0, [])
('NADH:quinone oxidoreductase\nsubunit F', 1935.0, [])
('formate dehydrogenase O\nsubunit &beta;', 1103.0, [])
('hydrogenase 2 small subunit', 901.0, [])
('hydrogenase 2 iron-sulfur\nprotein', 879.0, [])
('hydrogenase 2', 755.0, [])
('nitrate reductase A', 599.0, [])
('cytochr

In [10]:
# Get the base colors from Set2 palette
set2 = sns.color_palette("Set2")

def make_pastel(color, sat_reduction=0.3, light_increase=0.3):
    # Convert RGB to HSV
    hsv_color = mcolors.rgb_to_hsv(color)
    # Reduce saturation and increase lightness
    hsv_color[1] *= (1 - sat_reduction)  # reduce saturation
    hsv_color[2] += light_increase * (1 - hsv_color[2])  # increase lightness
    # Make sure lightness doesn't exceed 1
    hsv_color[2] = min(hsv_color[2], 1)
    # Convert back to RGB
    return mcolors.hsv_to_rgb(hsv_color)

def make_dark(color, sat_increase=0.3, light_decrease=0.3):
    # Convert RGB to HSV
    hsv_color = mcolors.rgb_to_hsv(color)
    # Reduce saturation and increase lightness
    hsv_color[1] *= (1 + sat_increase)  # reduce saturation
    hsv_color[2] += light_decrease * (1 + hsv_color[2])  # increase lightness
    # Make sure lightness doesn't exceed 1
    hsv_color[2] = max(hsv_color[2], 0)
    # Convert back to RGB
    return mcolors.hsv_to_rgb(hsv_color)

# Create a pastel version of the Set2 palette
pastel_set2 = [make_pastel(color) for color in set2]
dark_set2 = [make_dark(color) for color in set2]

In [15]:
plt.figure(figsize=(20, 20))
sunburst(data_transformed)